# Migrate Devices
Get the list of devices from UCM / backend systems. In our case, based on "site", we find devices owned or associated with a user and devices in the site specific device pools. 

Steps for migrating a UCM registered device:
1. verify the firmware is on the required load (set as default in UCM)
2. Add the device in Webex
3. Set load and cloud load server for the device
4. reboot device to get the new firmware
5. check status in Webex

In [ ]:
import os
import requests
import json        # just for pretty printing
from dotenv import load_dotenv

load_dotenv()
WEBEX_TOKEN = os.environ.get("WEBEX_TOKEN", 
                             # replace below if you don't have a .env
                             "NWVkODBmZWItMjM0Yy00ZDRlLWFjZmQtZWE4ZGNmN2Q1ZDVhOTFiY2UyYjQtNTNj_P0A1_0ec1bbe0-b5aa-454a-bb5b-aecc4e6bdb4c")
BASE_URI = "https://webexapis.com/v1"

headers = {
    "Authorization": "Bearer " + WEBEX_TOKEN, 
    "Content-Type": "application/json", 
    "Accept": "application/json"
}

### Adding a personal device
Assuming the user is properly configured, this can be done with a single call, based on MAC address

You will need the exact "model" string to be used, accepted ones can be found with API   
https://developer.webex.com/docs/api/v1/device-call-settings/read-the-list-of-supported-devices 

In [ ]:
data = {
    "mac": "00CCFC992E7C",
    "model": "DMS Cisco 8865",
    "personId": "Y2lzY29zcGFyazovL3VzL1BFT1BMRS9iYjZjNTQ4My00NGNkLTRjOGQtYWNlMS1mZGI4ZGQ1ZmQ4ZDE",    # this is Alice
}

try:
    response = requests.post(
        BASE_URI + "/devices",
        headers=headers,
        json=data
    )
    response.raise_for_status()

    print(f"{response.status_code} {response.reason}")
    device = response.json()
    print(json.dumps(device, indent=4))
except requests.exceptions.HTTPError as error:
    print(error)

### Adding infrastructure device (with hot desking)
For this, we will need to add a Workspace first with calling (assuming the number is added, see 'migrate_user')

In [ ]:
data = {
    "locationId": "Y2lzY29zcGFyazovL3VzL0xPQ0FUSU9OL2VlYmY5YjhmLTM5YTItNDE3Zi1hNWE4LWI5NzlkYzEyYTEzZQ",
    "displayName": "Desk-BRU-002",
    "type": "desk",
    "supportedDevices": "phones",
    "calling": {
        "type": "webexCalling",
        "webexCalling": {
            "phoneNumber": "+3227029922",
            "extension": "9922",
            "locationId": "Y2lzY29zcGFyazovL3VzL0xPQ0FUSU9OL2VlYmY5YjhmLTM5YTItNDE3Zi1hNWE4LWI5NzlkYzEyYTEzZQ",
        }
    }
}

try:
    response = requests.post(
        BASE_URI + "/workspaces",
        headers=headers,
        json=data
    )
    response.raise_for_status()

    print(f"{response.status_code} {response.reason}")
    new_workspace = response.json()
    print(json.dumps(new_workspace, indent=4))
except requests.exceptions.HTTPError as error:
    print(error)
    print(json.dumps(response.json(), indent=4))

In [ ]:
data = {
    "mac": "00CCFC992E7C",
    "model": "DMS Cisco 8865",
    "workspaceId": new_workspace["id"]
}

try:
    response = requests.post(
        BASE_URI + "/devices",
        headers=headers,
        json=data
    )
    response.raise_for_status()

    print(f"{response.status_code} {response.reason}")
    device2 = response.json()
    print(json.dumps(device2, indent=4))
except requests.exceptions.HTTPError as error:
    print(error)
    print(json.dumps(response.json(), indent=4))

### Reset Phone
Now that we configured the phone in webex, we can load the new firmware and reset. This will make the phone register to Webex, picking up the users or workspaces number 

In the below AXL call, we'll set the correct load and the loadServer, these might change, so better to check on the latest   
Note that you can also install this load on your UCM and not use the cloud loadServer

In [ ]:
url = "https://svluc-ucm-111.cisco.com/axl/"

headers = {
    "Content-Type": "text/xml; charset=utf-8",
    "SOAPAction": "CUCM:DB ver=12.0"
}

body = """
<ns0:Envelope xmlns:ns0="http://schemas.xmlsoap.org/soap/envelope/" xmlns:ns="http://www.cisco.com/AXL/API/12.0">
<ns0:Header/>
<ns0:Body>
  <ns:updatePhone>
    <name>SEP00279081B2F1</name>
    <loadInformation>sip8845_65.12-0-2MPP0001-116</loadInformation>
    <vendorConfig><loadServer>cloudupgrader.webex.com</loadServer></vendorConfig>"
  </ns:updatePhone>
</ns0:Body>
</ns0:Envelope>
"""

response = requests.post(url, headers=headers, data=body, auth=('username', 'pwd'))
print(response.content)

and with these set, we can reboot the phone so it picks up the new load 

In [ ]:
body = """
<ns0:Envelope xmlns:ns0="http://schemas.xmlsoap.org/soap/envelope/" xmlns:ns="http://www.cisco.com/AXL/API/12.0">
<ns0:Header/>
<ns0:Body>
  <ns:applyPhone>
    <name>SEP00279081B2F1</name>
  </ns:applyPhone>
</ns0:Body>
</ns0:Envelope>
"""

response = requests.post(url, headers=headers, data=body, auth=('ucername', 'pwd'))
print(response.content)

### check phone in Webex
The reset, download and install of the new firmware will take a while. So now we'll wait and periodically check on the status

In [ ]:
try:
    response = requests.get(
        BASE_URI + "/devices/Y2lzY29zcGFyazovL3VybjpURUFNOnVzLXdlc3QtMl9yL0NBTExJTkdfREVWSUNFLzAwY2NmYzk5LTJlN2MtNDU0YS1iYjViLWFlY2M0ZTZiZGI0Yw", 
        headers=headers
    )
    response.raise_for_status()
    
    print(f"{response.status_code} {response.reason}")
    print(json.dumps(response.json(), indent=4))
except requests.exceptions.HTTPError as error:
    print(error)